In [27]:
import rasterio
from matplotlib import pyplot as plt

# This notebook explores a single 4 band (blue, green, red, NIR) PlanetScope scene in a UTM projection.
image_file = "../week4/20160831_180302_0e26_3B_Visual.tif"

# Use Rasterio to open the image.
my_image = rasterio.open(image_file)
my_image

# We can take a quick look using the rasterio.plot.show function 
# This is a matplotlib shortcut for visualizing the raster 
from rasterio.plot import show
# show(my_image)

# So we can get our image bounds
xmin, ymin, xmax, ymax = my_image.bounds

# Let's get the x and y range first 
x_range = xmax - xmin
y_range = ymax - ymin
print(x_range, y_range)

coords = [
    (xmin,   ymin, (xmin + (x_range/2)), (ymin + (y_range/2))),
    (xmin,   (ymin + (y_range/2)), (xmin + (x_range/2)), ymax),
    
    ((xmin + (x_range/2)),   ymin, xmax, (ymin + (y_range/2))),
    ((xmin + (x_range/2)),   (ymin + (y_range/2)), xmax, ymax),
]

for coord in coords:

    xmin = coord[0]
    ymin = coord [1]
    xmax = coord[2]
    ymax = coord[3]

    my_geojson = [{
        "type": "Polygon", #let's define our geometry type, which as we have a square, is a polygon.
        "coordinates": [ #Here are our actual geometry coordinates
          [
            [
              xmin,
              ymin
            ],
            [
              xmax,
              ymin
            ],
            [
              xmax,
              ymax
            ],
            [
              xmin,
              ymax
            ],
            [
              xmin,
              ymin
            ]
          ],
        ]
      }]
    print(my_geojson)
    
    
    from rasterio.mask import mask

    # Now we can apply the rasterio mask 
    # We specify the function needs to crop (via crop=True)
    with rasterio.open(image_file) as img:
        clipped, transform = mask(img, my_geojson, crop=True)
        
    # Let's copy the metadata from the original ratserio object
    meta = my_image.meta.copy()

    # We will need to update this metadata, and provide the new clipped boundaries
    meta.update(
        {

            "transform": transform,
            "height":clipped.shape[1],
            "width":clipped.shape[2]
        }
    )

    filename_out = '{}.tif'.format(coord)

    # Finally, we can write new image to a GeoTIFF
    with rasterio.open(filename_out, 'w', **meta) as dst:
        dst.write(clipped)

    print('Writing complete')

28155.0 14052.0
[{'type': 'Polygon', 'coordinates': [[[623577.0, 4199985.0], [637654.5, 4199985.0], [637654.5, 4207011.0], [623577.0, 4207011.0], [623577.0, 4199985.0]]]}]
Writing complete
[{'type': 'Polygon', 'coordinates': [[[623577.0, 4207011.0], [637654.5, 4207011.0], [637654.5, 4214037.0], [623577.0, 4214037.0], [623577.0, 4207011.0]]]}]
Writing complete
[{'type': 'Polygon', 'coordinates': [[[637654.5, 4199985.0], [651732.0, 4199985.0], [651732.0, 4207011.0], [637654.5, 4207011.0], [637654.5, 4199985.0]]]}]
Writing complete
[{'type': 'Polygon', 'coordinates': [[[637654.5, 4207011.0], [651732.0, 4207011.0], [651732.0, 4214037.0], [637654.5, 4214037.0], [637654.5, 4207011.0]]]}]
Writing complete
